In [16]:
from settings import settings
from sqlalchemy import create_engine

db_string = f"postgresql://{settings.login}:{settings.haslo}@{settings.host}:{settings.port}/{settings.nazwa_bazy}"

db = create_engine(db_string)

connection_sqlalchemy = db.connect()

result_set = db.execute("SELECT * FROM city")  


In [17]:
# Ile kategorii filmów mamy w wypożyczalni?
result = db.execute("SELECT COUNT(*) FROM category")
print(result.all())


[(16,)]


In [18]:
# Wyświetl listę kategorii w kolejności alfabetycznej.
result = db.execute("SELECT name FROM category ORDER BY name ASC")
print(result.all())

[('Action',), ('Animation',), ('Children',), ('Classics',), ('Comedy',), ('Documentary',), ('Drama',), ('Family',), ('Foreign',), ('Games',), ('Horror',), ('Music',), ('New',), ('Sci-Fi',), ('Sports',), ('Travel',)]


In [19]:
# Znajdź najstarszy i najmłodszy film do wypożyczenia.
result = db.execute("(SELECT title, release_year FROM film ORDER BY release_year ASC LIMIT 1) \
    UNION ALL (SELECT title, release_year FROM film ORDER BY release_year DESC LIMIT 1)")
print(result.all())

[('Chamber Italian', 2006), ('Chamber Italian', 2006)]


In [20]:
# Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?
result = db.execute("SELECT COUNT(*) FROM rental WHERE rental_date BETWEEN '2005-07-01' AND '2005-08-01'")
print(result.all())


[(6709,)]


In [21]:
# Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?
result = db.execute("SELECT COUNT(*) FROM rental WHERE rental_date BETWEEN '2010-01-01' AND '2011-02-01'")
result.all()

[(0,)]

In [22]:
# Znajdź największą płatność wypożyczenia.
result = db.execute("SELECT amount FROM payment ORDER BY amount DESC LIMIT 1")
result.all()

[(Decimal('11.99'),)]

In [23]:
# Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.
result = db.execute("SELECT customer.first_name, customer.last_name, country.country FROM customer \
    LEFT JOIN address ON customer.address_id=address.address_id \
    LEFT JOIN city ON address.city_id=city.city_id \
    LEFT JOIN country ON city.country_id=country.country_id \
    WHERE country.country = 'Nigeria' OR country.country = 'Bangladesh' OR country.country = 'Poland'")
result.all()

[('Sonia', 'Gregory', 'Nigeria'),
 ('Brian', 'Wyman', 'Poland'),
 ('Sidney', 'Burleson', 'Poland'),
 ('Rodney', 'Moeller', 'Nigeria'),
 ('Stephen', 'Qualls', 'Bangladesh'),
 ('Velma', 'Lucas', 'Nigeria'),
 ('Marilyn', 'Ross', 'Nigeria'),
 ('Elsie', 'Kelley', 'Nigeria'),
 ('Gladys', 'Hamilton', 'Nigeria'),
 ('Frank', 'Waggoner', 'Bangladesh'),
 ('Marjorie', 'Tucker', 'Poland'),
 ('Carol', 'Garcia', 'Nigeria'),
 ('Leah', 'Curtis', 'Poland'),
 ('Ruben', 'Geary', 'Poland'),
 ('Olga', 'Jimenez', 'Nigeria'),
 ('Bertha', 'Ferguson', 'Nigeria'),
 ('Tracey', 'Barrett', 'Nigeria'),
 ('Jo', 'Fowler', 'Nigeria'),
 ('Johnnie', 'Chisholm', 'Poland'),
 ('Wallace', 'Slone', 'Nigeria'),
 ('Michelle', 'Clark', 'Bangladesh'),
 ('Russell', 'Brinson', 'Poland'),
 ('Jimmie', 'Eggleston', 'Poland'),
 ('Constance', 'Reid', 'Nigeria')]

In [24]:
# Gdzie mieszkają członkowie personelu?
result = db.execute("SELECT staff.first_name, staff.last_name, country.country, city.city, address.address FROM staff \
    LEFT JOIN address ON staff.address_id=address.address_id \
    LEFT JOIN city ON address.city_id=city.city_id \
    LEFT JOIN country ON city.country_id=country.country_id")
result.all()


[('Mike', 'Hillyer', 'Canada', 'Lethbridge', '23 Workhaven Lane'),
 ('Jon', 'Stephens', 'Australia', 'Woodridge', '1411 Lillydale Drive')]

In [25]:
# Ilu pracowników mieszka w Argentynie lub Hiszpanii?
result = db.execute("SELECT COUNT(*) FROM staff LEFT JOIN address ON staff.address_id=address.address_id")
result.all()

[(2,)]

In [26]:
# Jakie kategorie filmów zostały wypożyczone przez klientów?
result = db.execute("SELECT DISTINCT category.name FROM category \
    JOIN film_category USING(category_id)\
    JOIN film ON film_category.film_id=film.film_id\
    JOIN inventory ON inventory.film_id=film.film_id\
    JOIN rental ON rental.inventory_id=inventory.inventory_id\
    JOIN customer ON customer.customer_id=rental.customer_id")
result.all()

[('Sports',),
 ('Classics',),
 ('New',),
 ('Family',),
 ('Comedy',),
 ('Animation',),
 ('Travel',),
 ('Music',),
 ('Drama',),
 ('Horror',),
 ('Sci-Fi',),
 ('Games',),
 ('Documentary',),
 ('Foreign',),
 ('Action',),
 ('Children',)]

In [27]:
# Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.
result = db.execute("SELECT DISTINCT category.name FROM category \
    INNER JOIN film_category USING(category_id) \
    INNER JOIN film USING(film_id)\
    INNER JOIN inventory USING(film_id)\
    INNER JOIN rental USING(inventory_id)\
    INNER JOIN customer USING(customer_id)\
    INNER JOIN address USING(address_id)\
    INNER JOIN city USING(city_id)\
    INNER JOIN country USING(country_id)\
    WHERE country = 'United States'")
result.all()

[('Action',),
 ('Animation',),
 ('Children',),
 ('Classics',),
 ('Comedy',),
 ('Documentary',),
 ('Drama',),
 ('Family',),
 ('Foreign',),
 ('Games',),
 ('Horror',),
 ('Music',),
 ('New',),
 ('Sci-Fi',),
 ('Sports',),
 ('Travel',)]

In [28]:
# Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley
result = db.execute("SELECT film.title, actor.first_name, actor.last_name FROM film \
    RIGHT JOIN film_actor ON film_actor.film_id=film.film_id \
    RIGHT JOIN actor ON actor.actor_id=film_actor.actor_id \
    WHERE actor.first_name='Olympia' AND actor.last_name='Pfeiffer' \
    OR actor.first_name='Julia' AND actor.last_name='Zellweger' \
    OR actor.first_name='Ellen' AND actor.last_name='Presley'")
print(len(result.all()))

69
